In [5]:
import pandas as pd
import plotly.express as px
import numpy as np

In [9]:


def load_profitable_data(year: str = "2030"):
    return pd.read_csv(f"../data_final/{year}_profitable.csv", index_col=0)


def load_all_data(year: str = "2030"):
    return pd.read_csv(f"../data_final/{year}.csv", index_col=0)




def visualise_profitability(df):
    fig = px.scatter_mapbox(
        df,
        lat="latitude",
        lon="longitude",
        zoom=5,
        height=800,
        width=800,
        color="profitability threshold",
        color_continuous_scale="viridis",
        size=df.size_station.map({"small": 6, "medium": 12, "large": 18}),
    )
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    return fig

In [69]:
def compute_pareto(h2):
    return h2.cumsum() / h2.sum(), h2.sum()

def load_and_clean(year="2030"):
    # cutoff = {"2030": 30, "2040": 160}
    # scenarios = {"monopoly": 1, "oligopoly": 2}
    # scenario_cutoff = cutoff[year] * scenarios[scenario]
    df = load_all_data(year).sort_values(by=["prioritize", "total_h2_sold_per_station"], ascending=False)
    df["cumulative demand"], total_demand = compute_pareto(df.total_h2_sold_per_station)
    df.reset_index(inplace=True, drop=True)
    return df, total_demand

def add_construction_year(df, construction_rate: int = 5, scenario="monopoly"):
    start_year = {5: 2024, 13: 2030, 3:2030}
    scenario_factor = {"monopoly": 1, "oligopoly": 2, "latecomer": 1}
    total_building_rate = construction_rate * scenario_factor[scenario]
    df["construction_year"] = (
        np.floor(df.index / total_building_rate) + start_year[construction_rate]
    )
    df["construction_year"] = df["construction_year"].astype(int)


def keep_2030_stations(df_2030, df_2040):
    cutoff = df_2030.shape[0]
    return pd.concat([df_2030, df_2040.iloc[:-cutoff, :]])


def pareto_with_2030_stations(df_2040, total_demand=None):
    df = df_2040.copy()
    df.sort_values(by=["prioritize", "total_h2_sold_per_station"], ascending=False).reset_index(inplace=True)
    if total_demand:
        df["cumulative demand"] = df.total_h2_sold_per_station.cumsum() / total_demand
    else:
        df["cumulative demand"] = df.total_h2_sold_per_station.cumsum() / df.total_h2_sold_per_station.sum()
    return df, df.total_h2_sold_per_station.sum()


df_2030, total_demand_2030 = load_and_clean()
df_2040, _ = load_and_clean("2040")
add_construction_year(df_2030, scenario="oligopoly")
add_construction_year(df_2040, construction_rate=13, scenario="oligopoly")
merged_2040 = keep_2030_stations(df_2030.iloc[:60], df_2040)


In [11]:
(merged_2040.total_h2_sold_per_station.sum() - total_demand) / total_demand

NameError: name 'total_demand' is not defined

In [13]:
total_demand

814845.44

In [12]:
final_2040, total_demand_2040 = pareto_with_2030_stations(merged_2040)
final_2040 = final_2040.iloc[:320]

In [13]:
final_2040

,total_h2_sold_per_station,prioritize,profitability threshold,longitude,latitude,station_location,size_station,cumulative demand,construction_year
0,4390.400000,1,1.097600,-1.723687,47.227908,hub,large,0.005010,2024
1,3440.640000,1,0.860200,4.806896,43.513768,hub,large,0.008937,2024
2,3269.873845,1,0.817468,2.871695,50.101134,road,large,0.012669,2024
3,3269.873845,1,0.817468,2.900112,50.314659,road,large,0.016400,2024
4,2525.279888,1,0.631320,4.797245,44.671062,road,large,0.019282,2024
...,...,...,...,...,...,...,...,...,...
255,788.480000,0,0.788500,2.399125,47.081166,hub,small,0.994703,2039
256,788.480000,0,0.788500,4.304100,48.937627,hub,small,0.995603,2039
257,788.480000,0,0.788500,2.157250,50.234095,hub,small,0.996503,2039
258,788.480000,0,0.788500,6.861796,47.507832,hub,small,0.997403,2039


In [14]:
total_demand_2040

876257.7037511724

In [15]:
df_2030 = pd.read_csv("../oligopoly_2030.csv", index_col=0)
df_2040 = pd.read_csv("../oligopoly_2040.csv", index_col=0)

In [16]:
total_demand_2030

122343.68

In [17]:
propriety = 0
up_to_year = 2029
ours = df_2030[(df_2030.propriety == propriety) & (df_2030.construction_year <= up_to_year)]
ours = ours.copy()
ours.drop("cumulative_demand", axis=1, inplace=True)
ours["cumulative_demand"] = ours.total_h2_sold_per_station.cumsum() / total_demand_2030

In [18]:
ours.iloc[:30]

,total_h2_sold_per_station,prioritize,profitability threshold,longitude,latitude,station_location,size_station,construction_year,propriety,cumulative_demand
1,3440.640000,1,0.860200,4.806896,43.513768,hub,large,2024,0,0.028123
2,3269.873845,1,0.817468,2.871695,50.101134,road,large,2024,0,0.054850
4,2525.279888,1,0.631320,4.797245,44.671062,road,large,2024,0,0.075491
5,2525.279888,1,0.631320,5.229411,43.557966,road,large,2024,0,0.096131
7,1953.280000,1,0.976600,1.822648,48.992630,hub,medium,2024,0,0.112097
8,1899.520000,1,0.949800,0.196785,48.007385,hub,medium,2024,0,0.127623
9,1833.448493,1,0.916724,3.659101,47.796680,road,medium,2024,0,0.142609
11,1756.160000,1,0.878100,2.744224,49.085962,hub,medium,2025,0,0.156963
12,1738.240000,1,0.869100,-0.500972,43.891132,hub,medium,2025,0,0.171171
14,1727.739860,1,0.863870,6.106938,49.362415,road,medium,2025,0,0.185293


In [19]:
ours.iloc[]

SyntaxError: invalid syntax (3451256071.py, line 1)

In [143]:
def load_s3_data(year: str = "2030", property="ours"):
    df = pd.read_csv(f"../data_final/scenario3_{year}.csv", index_col=0)
    return df.reset_index(drop=True)


def scenario3_compute_cumdemand(df, total):
    df.total_h2_sold_per_station.apply(lambda x: 0 if x == "na" else x)
    df.total_h2_sold_per_station.astype(float)
    df["cumulative_demand"] = df.total_h2_sold_per_station.cumsum() / total


In [144]:
s3_2030 = load_s3_data("2030")
scenario3_compute_cumdemand(s3_2030, total_demand_2030)

s3_2040 = load_s3_data("2040")
scenario3_compute_cumdemand(s3_2040, total_demand_2040)


TypeError: can only concatenate str (not "int") to str

In [142]:
s3_2030

,total_h2_sold_per_station,prioritize,profitability threshold,longitude,latitude,station_location,size_station,cumulative_demand,property,nearby_total_station,new_h2_demand,new_profitability_threshold,new_profitability
0,976.78628886874,1.0,0.97678628886874,0.160167,46.211480,road,small,0.8593655836418589,ours,1.0,976.78628886874,0.9767862888687401,profitable
1,976.78628886874,1.0,0.97678628886874,-0.273450,45.240433,road,small,0.8776634088459595,ours,1.0,976.78628886874,0.9767862888687401,profitable
2,917.8053469550974,1.0,0.9178053469550974,2.119418,47.208008,road,small,0.9485667036857968,ours,1.0,917.8053469550974,0.9178053469550974,profitable
3,917.8053469550974,1.0,0.9178053469550974,3.160584,45.843331,road,small,0.965759657765264,ours,1.0,917.8053469550974,0.9178053469550974,profitable
4,na,na,na,2.128150,49.409445,na,na,na,total,na,na,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,na,na,na,3.528223,47.880725,na,na,na,total,na,na,na,na
328,na,na,na,-1.846042,47.927613,na,na,na,total,na,na,na,na
329,na,na,na,-1.033310,48.937190,na,na,na,total,na,na,na,na
330,na,na,na,-3.175557,48.226819,na,na,na,total,na,na,na,na


In [109]:
s3_2040["exact_position"] = s3_2040.apply(lambda x: str(x.longitude) + str(x.latitude), axis=1)
s3_2030["exact_position"] = s3_2030.apply(lambda x: str(x.longitude) + str(x.latitude), axis=1)

add_construction_year(s3_2030, construction_rate=5, scenario="latecomer")
add_construction_year(s3_2040, construction_rate=3, scenario="latecomer")

In [112]:
new_s3_2040 = pd.concat([s3_2030, s3_2040])
new_s3_2040.drop_duplicates(subset="exact_position")

,total_h2_sold_per_station,prioritize,profitability threshold,longitude,latitude,station_location,size_station,cumulative_demand,property,nearby_total_station,new_h2_demand,new_profitability_threshold,new_profitability,exact_position,construction_year
0,976.78628886874,1.0,0.97678628886874,0.160167,46.211480,road,small,0.007984,ours,1.0,976.78628886874,0.9767862888687401,profitable,0.160167134567160246.2114804408092,2024
1,976.78628886874,1.0,0.97678628886874,-0.273450,45.240433,road,small,0.015968,ours,1.0,976.78628886874,0.9767862888687401,profitable,-0.273449679239427345.24043326446728,2024
2,917.8053469550974,1.0,0.9178053469550974,2.119418,47.208008,road,small,0.023470,ours,1.0,917.8053469550974,0.9178053469550974,profitable,2.119417653692501347.20800802246639,2024
3,917.8053469550974,1.0,0.9178053469550974,3.160584,45.843331,road,small,0.030972,ours,1.0,917.8053469550974,0.9178053469550974,profitable,3.16058396195734745.8433306210893,2024
0,25141.76,1.0,6.285399999999999,5.004510,49.445610,hub,large,0.028692,ours,1.0,25141.76,6.2854399999999995,profitable,5.0045149.44561,2030
1,18565.12,1.0,4.6413,4.895245,44.031628,hub,large,0.049879,ours,6.0,3094.1866666666665,0.7735466666666666,profitable,4.8952447244.03162786,2030
2,10805.76,1.0,2.7014,1.656156,48.330494,hub,large,0.062211,ours,2.0,5402.88,1.35072,profitable,1.656155648.3304943,2030
3,8171.52,1.0,2.0429,2.476574,49.264898,hub,large,0.071536,ours,3.0,2723.84,0.68096,profitable,2.4765742549.26489755,2031
4,7490.559999999999,1.0,1.8726,-1.049078,47.257417,hub,large,0.080085,ours,1.0,7490.559999999999,1.8726399999999996,profitable,-1.049077729567798247.2574172,2031
5,7239.679999999999,1.0,1.8099,4.928726,43.397391,hub,large,0.088347,ours,3.0,2413.2266666666665,0.6033066666666667,profitable,4.928726543.3973905,2031
